In [3]:
import json
import torch
import os, sys, random, numpy, pickle
import argparse
import logging
from pprint import pprint
from qdgat.drop_reader import DropReader
from qdgat.drop_dataloader import DropBatchGen
from qdgat.network import QDGATNet
from qdgat.utils import AverageMeter
from datetime import datetime
from qdgat.optimizer import AdamW
from qdgat.utils import create_logger
from transformers import RobertaTokenizer, RobertaModel, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler
from qdgat.drop_dataloader import create_collate_fn
# from tqdm.notebook import tqdm
from tqdm import tqdm

In [4]:
logger = logging.getLogger()
logging.basicConfig(level = logging.INFO)
formatter = logging.Formatter(fmt='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S')
ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
logger.addHandler(ch)

In [5]:
from collections import namedtuple

args = {
    'passage_length_limit': 463,
    'question_length_limit': 46,
    'data_dir': "./raw_data",
    'roberta_model': "roberta-base",
    'output_dir': './data'
}

args = namedtuple("Arguments", args.keys())(*args.values())

In [6]:
tokenizer = RobertaTokenizer.from_pretrained(args.roberta_model)

In [7]:
def preprocess_drop(args, data_path, tokenizer, mode):
    if not os.path.exists(data_path):
        raise Exception("Missing %s for preprocessing."%data_path)

    skip_when_all_empty = ["passage_span", "question_span", "addition_subtraction", "counting", "multi_span"] if mode=='train' else None
    reader = DropReader(
        tokenizer, args.passage_length_limit, args.question_length_limit,
        skip_when_all_empty=skip_when_all_empty
    )
    data = reader._read(data_path)
    return data

In [8]:
# pre-process the data for pretrained model
pretrained_data = preprocess_drop(args, os.path.join(args.data_dir, "pretrain_split.json"), tokenizer, 'train')
cache_fpath = os.path.join(args.output_dir, "%s.pkl"%'pretrain')
with open(cache_fpath, "wb") as f:
    pickle.dump(pretrained_data, f)

Reading file at %s ./raw_data/pretrain_split.json
Reading the dataset
skip: 54dc13ba-37d7-4d2e-bcd0-0a6d3aba8fac: How many people are there per 2 square miles ?
skip: da4758d7-6054-44c8-8c2c-93172182689e: How many people are there per 2 square miles ?
skip: f9bfb86b-ae5b-4553-83ac-416873b1b664: How many more swine than cattle are there in Italy ?
skip: 7fe7bef3-e846-4927-a1f3-493a489cdf30: When did Whitten first become a representative of the 1st District ?
skip: 6234e8e5-5b46-4e03-8f20-85c5da1f5e82: How many total votes were cast in the primary ?
skip: 07067a4a-243c-424d-9bc6-8a20fb57ce13: How many more votes did the four remaining candidate get than Fox and Whitten combined ?
skip: 87040c16-7170-429e-99bc-65a97320e290: How many more votes did the top three candidates get than the bottom three candidates ?
skip: 6a2b1aaf-9635-4f14-8bcf-4628721def48: Which kicker missed a field goal of over 50 yards ?
skip: b89bc494-3c1f-4562-9005-4c2cae78df48: What BMI's have been found to have the lo

In [16]:
data = preprocess_drop(args, os.path.join(args.data_dir, 'cv_fold-0.json'), tokenizer, mode='train')

Reading file at %s ./raw_data/cv_fold-0.json
Reading the dataset
skip: 160b7985-d826-4ff6-8006-d8d4c77f96a2: How many days passed between John entering Perpignan and the treaty at Perpignan ?
skip: 67da715c-c057-4d09-85b7-3c6649b024b1: How many men were on each ship that left on July 8 , 1497 , on average ?
skip: 7b139d33-6d75-4fb1-aae6-5b84ec36edd2: How many days passed between the fleet leaving Libon and them making first contact ?
skip: 6b754ab9-9e48-4c6d-ae48-69d81bb71903: How many months did these events span for ?
skip: 326e4d80-25c9-440a-9bff-3603bfd9a345: How many years after the Dow was holding steady between 7,000 and 9,000 did the Dow pass 14,000 ?
skip: 09e97b73-afdf-483d-bb44-9569a34b0e27: Around how many months do these events span ?
skip: b619bf65-97f5-419d-a38b-36ce7dccc383: How many dollars would Johnson have earned in 2008 if he achieved all bonuses ?
skip: e39c2efe-9004-4225-80f0-3e7aa9fc2552: How many percent of people were not working at home ?
skip: e2d4beeb-2b95-